In [1]:
### Basic Dependency stuff.
import sys
!pip install --prefix {sys.prefix} psycopg2 pandas faker

import psycopg2
import time
import sys, math

In [23]:
### Defining a few SQL Statements, including a partitioned SQL table

drop_if_exists = """DROP TABLE  IF EXISTS public.employee"""

employee_ddl = """CREATE TABLE public.employee (
    id int8 NOT NULL,
    name varchar(120) NOT NULL,
    salary int8 NOT NULL
) PARTITION BY RANGE (salary);

CREATE TABLE public.employee_to_1500 PARTITION OF public.employee
    FOR VALUES FROM (0) TO (1500);
    
CREATE TABLE public.employee_to_3000 PARTITION OF public.employee
    FOR VALUES FROM (1500) TO (3000);
    
CREATE TABLE public.employee_to_6000 PARTITION OF public.employee
    FOR VALUES FROM (3000) TO (6000);
"""


sql_dummy_data = """WITH salary_list AS (
    SELECT '{1000, 2000, 5000}'::INT[] salary
)
INSERT INTO public.employee
(id, name, salary)
SELECT n, 'Employee ' || n as name, salary[1 + mod(n, array_length(salary, 1))]
FROM salary_list, generate_series(1, 10000) as n"""


In [24]:
conn = psycopg2.connect(host="postgres", port = 5432, database="demo_catalog", user="admin", password="password")
cur = conn.cursor()
  
############--------#--------#--------#--------#--------#--------#--------#--------#--------#--------#--------#-----
## Delete possible old table, create the new one, add a bunch of columns, get ready!

cur.execute(drop_if_exists)
cur.execute(employee_ddl)
cur.execute(sql_dummy_data)

cur.execute(f"""SELECT * FROM public.employee LIMIT 10""")
print(cur.fetchall())
cur.execute(f"""SELECT * FROM public.employee_to_6000 LIMIT 10""")
print(cur.fetchall())

[(3, 'Employee 3', 1000), (6, 'Employee 6', 1000), (9, 'Employee 9', 1000), (12, 'Employee 12', 1000), (15, 'Employee 15', 1000), (18, 'Employee 18', 1000), (21, 'Employee 21', 1000), (24, 'Employee 24', 1000), (27, 'Employee 27', 1000), (30, 'Employee 30', 1000)]
[(2, 'Employee 2', 5000), (5, 'Employee 5', 5000), (8, 'Employee 8', 5000), (11, 'Employee 11', 5000), (14, 'Employee 14', 5000), (17, 'Employee 17', 5000), (20, 'Employee 20', 5000), (23, 'Employee 23', 5000), (26, 'Employee 26', 5000), (29, 'Employee 29', 5000)]


**Task**: So what happens, if we insert something with a larger salary?

In [27]:
sql_dummy_data = """INSERT INTO public.employee
(id, name, salary) VALUES (1, 'rich_bob', 7000)"""
cur.execute(sql_dummy_data)


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


That's not cool right? The problem is, **partitioning** is for some reason I haven't figured out, in almost all tools 
like Postgres and other object stores which are topped of with Hive something almost completely
manual. Just think about the next task...

**Task**: So let's change the partition key because right now we realize the salary is all in the latter bucket, 
we might from now on want to partition by employee name.

**How does that work in PostgreSQL?** Mostly by hand and rewriting the old partitions. You could hack around it
and create "new partitions just for newly inserted data" by using triggers on read & write, but it involves
quite a bit of manual effort. 


**Using Hive**: If you're using most object stores with just Hive on top, you'll need to migrate the data and rewrite 
the old data sets into new partitions (or as explained, do some manual workarounds based on insert time). 
=> No Fun!

### Partition Changes in Iceberg: Hidden Magic ###

In [10]:
spark

In [11]:
%%sql

DROP TABLE IF EXISTS nyc.employee

++
||
++
++

In [12]:
employee_ddl = """CREATE TABLE nyc.employee (
    id int,
    name varchar(120),
    salary int
) PARTITIONED BY (truncate(1500,salary));""" ## using a partition transform function to create partitions 
## just as above where each bucket has salaries of "width 1500" => high-low = 1500 roughly.

spark.sql(employee_ddl)

DataFrame[]

In [13]:
%%sql

SELECT * FROM nyc.employee.partitions

partition,record_count,file_count


In [14]:
############--------#--------#--------#--------#--------#--------#--------#--------#--------#--------#--------#-----
## Some preparation data & SQLs

data = [(1, "Bob",1200)]
for i in range(0,1000):
    data.append((2,f"Bob{i}",i*7))
print(f"Created {len(data)} data rows to insert")

print(f"data is of size: {sys.getsizeof(data)/102400} MB ")
rdd = spark.sparkContext.parallelize(data, numSlices=math.ceil(sys.getsizeof(data) / 102400))

df = rdd.toDF()
columns = ["id","name","salary"]
df = rdd.toDF(columns)
df.write.mode("append").saveAsTable("nyc.employee")
# write takes about 25 secs and is 10 times the size of the other write... 


spark.read.format("iceberg").load("nyc.employee").show(truncate = False)

Created 1001 data rows to insert
data is of size: 0.086484375 MB 
+---+-----+------+
|id |name |salary|
+---+-----+------+
|1  |Bob  |1200  |
|2  |Bob0 |0     |
|2  |Bob1 |7     |
|2  |Bob2 |14    |
|2  |Bob3 |21    |
|2  |Bob4 |28    |
|2  |Bob5 |35    |
|2  |Bob6 |42    |
|2  |Bob7 |49    |
|2  |Bob8 |56    |
|2  |Bob9 |63    |
|2  |Bob10|70    |
|2  |Bob11|77    |
|2  |Bob12|84    |
|2  |Bob13|91    |
|2  |Bob14|98    |
|2  |Bob15|105   |
|2  |Bob16|112   |
|2  |Bob17|119   |
|2  |Bob18|126   |
+---+-----+------+
only showing top 20 rows



In [15]:
%%sql

SELECT * FROM nyc.employee.partitions

partition,record_count,file_count
Row(salary_trunc=3000),214,1
Row(salary_trunc=1500),214,1
Row(salary_trunc=0),216,1
Row(salary_trunc=6000),142,1
Row(salary_trunc=4500),215,1


In [31]:
%%sql 

INSERT INTO nyc.employee
(id, name, salary) VALUES (1, 'rich_bob', 7000)

/* works like a charm */

++
||
++
++

**Task**: Changing the partition, as you might imagine, is just as easy in iceberg. Let's do that!

In [33]:
%%sql

ALTER TABLE nyc.employee ADD PARTITION FIELD truncate(4,name)

++
||
++
++

In [34]:
############--------#--------#--------#--------#--------#--------#--------#--------#--------#--------#--------#-----
## Some preparation data & SQLs

data = [(1, "Bob",1200)]
for i in range(0,100):
    data.append((2,f"Bob{i}",i*7))
print(f"Created {len(data)} data rows to insert")

rdd = spark.sparkContext.parallelize(data, numSlices=1)

df = rdd.toDF()
columns = ["id","name","salary"]
df = rdd.toDF(columns)
df.write.mode("append").saveAsTable("nyc.employee")

Created 101 data rows to insert


In [35]:
%%sql

SELECT * FROM nyc.employee.partitions
/* We still got old partitions with name_trunc_4 set to None, but we 

*/ 

partition,record_count,file_count
"Row(salary_trunc=6000, name_trunc_4=None)",144,3
"Row(salary_trunc=0, name_trunc_4='Bob0')",1,1
"Row(salary_trunc=4500, name_trunc_4=None)",215,1
"Row(salary_trunc=0, name_trunc_4='Bob')",1,1
"Row(salary_trunc=0, name_trunc_4='Bob4')",11,1
"Row(salary_trunc=0, name_trunc_4='Bob3')",11,1
"Row(salary_trunc=0, name_trunc_4='Bob2')",11,1
"Row(salary_trunc=0, name_trunc_4='Bob1')",11,1
"Row(salary_trunc=0, name_trunc_4='Bob8')",11,1
"Row(salary_trunc=0, name_trunc_4='Bob7')",11,1
